LOAD THE MODULES

In [ ]:
import os

SET TIDE PATH AND TIDE LIB

In [ ]:
TIDE_PATH="/media/susmita/b3b9571f-7454-4bd3-93df-d81f26409fcc/tide/TiDE/"
TIDE_LIB="/media/susmita/b3b9571f-7454-4bd3-93df-d81f26409fcc/tide/TiDE/src/.libs"

RUNNING TIDEPY TO GENERATE THE LIGHTCURVES AND SAVING TIME AND LUMINOSITY FOR EVERY LIGHT CURVE

In [ ]:
import tidepy
import numpy as np
import matplotlib.pyplot as plt

p = tidepy.Parameters()

def light_curve_in_mass_range(masses):
    p = tidepy.Parameters()
    lc_calculator = tidepy.Light_curve_of_tde(p)
    results = []
    for M6 in masses:
        lc_calculator.PythonPar.bh_M6 = M6
        lc_calculator.PythonPar.param_init()
        results.append(lc_calculator.light_curve())
    return results

massgrid = np.arange(0.1, 1.505, 0.1)

allcurve = light_curve_in_mass_range(massgrid)

plt.figure(dpi=150, tight_layout=True)
for mass, crv in zip(massgrid, allcurve):
    time = crv[0]
    luminosity = crv[1] * p.nu
    print(time)
    print(luminosity)
    
    # Save time and luminosity to a file for each mass
    filename = f"light_curve_M6_{mass:.1f}.txt"
    np.savetxt(filename, np.column_stack((time, luminosity)), header="Time [d]   Luminosity [erg/s]", fmt="%.6e")
    
    plt.plot(time, luminosity, label=f'$M_6={mass:.1f}\\,M_\\odot$')

plt.ylim(bottom=1e39)
plt.yscale('log')
plt.legend(bbox_to_anchor=[1.1, 1])
plt.xlabel('time [d]')
plt.ylabel(r'$\nu L_\nu$ [erg/s]')
plt.show()


CONVERT LUMINOSITY TO MAGNITUDE

In [ ]:
import numpy as np

# Constants
d = 1  # Distance in cm
midpoint = 7556.09  # Filter point in Hz

# Function to calculate the magnitude from luminosity
def luminosity_to_magnitude(L):
    flux= L/(4.0*np.pi*d*d*3.086e+24*3.086e+24) ####d in Mpc
    print(flux)
    nu= 2.99792458e18/midpoint
    print(nu) #### midpoint in angstrom
    fnu=flux/nu
    #### convert to Jy
    fnu_jy=1e23*fnu
    print(fnu_jy)
    m =  -2.5*np.log10(fnu_jy/3631)
    return m 

# Read the input file
input_file = "light_curve_M6_0.1.txt"
data = np.loadtxt(input_file, skiprows=1)

# Time and luminosity from the data
time = data[:, 0]  # Time [d]
luminosity = data[:, 1]  # Luminosity [erg/s]

# Convert luminosity to magnitude
magnitudes = np.array([luminosity_to_magnitude(L) for L in luminosity])

# Save the time and magnitudes to a new file
output_file = "test_converted_magnitudes.dat"
np.savetxt(output_file, np.column_stack((time, magnitudes)), header="Time [d] Magnitude", fmt="%10.6f %10.6f")

print(f"Data has been converted and saved to {output_file}")


GENERATING THE LIGHTCURVES USING RUBIN SIM

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rubin_sim.maf as maf
from rubin_scheduler.data import get_baseline
from rubin_sim.maf.slicers import UserPointsSlicer
from rubin_sim.phot_utils import DustValues
import random
from astropy.timeseries import LombScargle
import P4J

# Custom Metric Class
class MyCustomPSDMetric(maf.BaseMetric):
    def __init__(self, event_id, mjd_col='observationStartMJD', ebv=0.0, distance=10.0, **kwargs):
        self.event_id = event_id
        self.mjd_col = mjd_col
        self.ebv = ebv
        self.distance = distance
        dust_properties = DustValues()
        self.ax1 = dust_properties.ax1
        
        self.filters = list(self.ax1.keys())
        
        self.lsst_single_epoch_depth = {
            "u": 23.9, "g": 25.0, "r": 24.7, "i": 24.0, "z": 23.3, "y": 22.1,
        }

        self.filter_colors = {
            "u": "blue", "g": "green", "r": "red", "i": "orange", "z": "purple", "y": "brown",
        }
        
        super().__init__(col=[mjd_col], units='Arbitrary', **kwargs)
        
    def run(self, data_slice, slice_point=None):
        times = data_slice[self.mjd_col]
        print("Observed Times (MJD):")
        print(len(times))  # Debugging: Print observed times

        lightcurve_file = 'test_converted_magnitudes.dat'  # Read in the lightcurve data
        lc_data = np.loadtxt(lightcurve_file, skiprows=1)

        model_times1 = lc_data[:, 0]  # Time
        print(model_times1)

        model_values = lc_data[:, 1]  # Flux

        # Change to MJD for easier use
        start_mjd = 60800
        model_times = start_mjd + model_times1

        # Debugging: Print adjusted model times
        print("Simulated Model Times (adjusted to MJD):")
        print(model_times)

        # Check the alignment of observed times with simulated model times
        print("Observed Times vs. Simulated Model Times:")
        for observed_time in times:
            closest_model_time = model_times[np.abs(model_times - observed_time).argmin()]
            print(f"Observed Time: {observed_time}, Closest Model Time: {closest_model_time}")

        # Getting the corresponding magnitudes for the specified slice
        indices = np.searchsorted(model_times, times)
        indices = np.clip(indices, 0, len(model_times1) - 1)
        corresponding_magnitudes = model_values[indices]
        sorted_indices = np.argsort(times)
        sorted_times = times[sorted_indices]
        sorted_magnitudes = corresponding_magnitudes[sorted_indices]

        # File saving logic (same as before)...
        ra, dec = slice_point['ra'], slice_point['dec']
        
        base_filename = f'lc_ra{ra:.2f}_dec{dec:.2f}_event{self.event_id}'
        lightcurve_filename = f'{base_filename}.dat'
        model_filename = f'{base_filename}_model.dat'
        sorted_filename = f'{base_filename}_sorted.dat'
        obs_mag_filename = f'{base_filename}_obsmag.dat'

        header = (f'time flux\n')
        np.savetxt(obs_mag_filename, np.column_stack((sorted_times, sorted_magnitudes)), header=header, comments='')
        
        # Saving model data before any filtering
        header = (f'time flux\n')
        np.savetxt(model_filename, np.column_stack((model_times, model_values)), header=header, comments='')

        # NUMBER OF DAYS TO SIMULATE
        num_days_to_simulate = 100
        mask = (sorted_times >= sorted_times.min()) & (sorted_times <= sorted_times.min() + num_days_to_simulate)
        times, model_values = sorted_times[mask], sorted_magnitudes[mask]

        # Saving the filtered and sorted times and model_values to a file after applying num of days to simulate
        header = (f'time flux\n')
        np.savetxt(lightcurve_filename, np.column_stack((times, model_values)), header=header, comments='')

        self.plot_lightcurve_only(times, model_values, ra, dec, lightcurve_filename)
        self.plot_model_and_lightcurve(times, model_values, ra, dec, lightcurve_filename)
        
        return np.mean(model_values)
    
    def plot_lightcurve_only(self, times, model_values, ra, dec, filename):
        extinction_coefficients = {
            'u': 4.8, 'g': 3.6, 'r': 2.7, 'i': 2.05, 'z': 1.5, 'y': 1.3
        }

        for filtername in self.filters:
            plt.figure()

            # Apply extinction correction (no magnitude conversion)
            a_x = extinction_coefficients[filtername] * self.ebv
            values_with_extinction = model_values + a_x + 5 * np.log10(self.distance * 1.0e3) - 5.0

            # Directly use the flux (no conversion to magnitude)
            mask = (values_with_extinction < self.lsst_single_epoch_depth[filtername]) & (values_with_extinction > 0)
            times_filtered, flux_filtered = times[mask], values_with_extinction[mask]

            filter_filename = filename.replace('.dat', f'_filter_{filtername}.dat')
            np.savetxt(filter_filename, np.column_stack((times_filtered, flux_filtered)),
                       header='time flux', comments='')

            plt.plot(times_filtered, flux_filtered, 'o', markersize=2, color=self.filter_colors[filtername], 
                     label=f'{filtername} filter')

            plt.xlabel('Time (MJD)')
            plt.ylabel('Flux')
            plt.title(f'Lightcurve for {filtername} filter, RA: {ra:.2f}, Dec: {dec:.2f}')
            plt.legend()
            plt.savefig(filename.replace('.dat', f'_lightcurve_{filtername}.png'))
            plt.close()

    def plot_model_and_lightcurve(self, times, model_values, ra, dec, filename):
        for filtername in self.filters:
            plt.figure()
            plt.plot(times, model_values, 'o', markersize=2, label='Model Lightcurve (no extinction)')

            # Apply extinction correction (no magnitude conversion)
            a_x = self.ax1[filtername] * self.ebv
            values_with_extinction = model_values + a_x + 5 * np.log10(self.distance * 1.0e3) - 5.0

            # Directly use the flux (no conversion to magnitude)
            mask = (values_with_extinction < self.lsst_single_epoch_depth[filtername]) & (values_with_extinction > 0)
            times_filtered, flux_filtered = times[mask], values_with_extinction[mask]

            plt.plot(times_filtered, flux_filtered, 'x', markersize=2, color=self.filter_colors[filtername],
                 label=f'{filtername} filter')

            plt.xlabel('Time (MJD)')
            plt.ylabel('Flux')
            plt.title(f'Model and Lightcurve for {filtername} filter, RA: {ra:.2f}, Dec: {dec:.2f}')
            plt.legend()
            plt.savefig(filename.replace('.dat', f'_model_lightcurve_{filtername}.png'))
            plt.close()

def main():
    opsim_fname = get_baseline()
    runName = os.path.splitext(os.path.basename(opsim_fname))[0]

    df = pd.read_csv('params.csv')

    num_events_to_generate = 1
    bundles_dict = {}

    for i in range(num_events_to_generate):
        row = df.iloc[i % len(df)]
        ra, dec = row['ra'], row['dec']
        print(ra)
        print(dec)
        ebv, distance = row['ebv'], row['distance']
        event_id = i + 1
        

        psd_metric = MyCustomPSDMetric(event_id=event_id, ebv=ebv, distance=distance)
        test_slicer = maf.UserPointsSlicer(ra, dec)
        bundle = maf.MetricBundle(psd_metric, test_slicer, None, run_name=runName)
        bundles_dict[f'psd_lightcurve_{event_id}'] = bundle

    g = maf.MetricBundleGroup(bundles_dict, opsim_fname, out_dir='test', results_db=None)
    g.run_all()

if __name__ == "__main__":
    main()


TESTING WHETHER THE MODEL AND THE DATA IS ACTUALLY SIMILAR TIME OR NOT

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load data from the first file (e.g., 'u_magnitude_time.dat')
data1 = np.loadtxt('lc_ra4.66_dec-0.36_event1_model.dat', comments='#', skiprows=1)
time1 = data1[:, 0]
magnitude1 = data1[:, 1]

# Load data from the second file (e.g., 'g_magnitude_time.dat')
data2 = np.loadtxt('lc_ra4.66_dec-0.36_event1.dat', comments='#', skiprows=1)
time2 = data2[:, 0]
magnitude2 = data2[:, 1]

# Load data from the first file (e.g., 'u_magnitude_time.dat')
data3 = np.loadtxt('u_magnitude_time.dat', comments='#', skiprows=1)
time3 = data3[:, 0]
magnitude3 = data3[:, 1]


# Create the plot
plt.figure(figsize=(10, 6))

# Plot the first dataset
plt.plot(time1, magnitude1, label='model', marker='o', linestyle='-', color='b')

# Plot the second dataset
plt.plot(time2, magnitude2, label='data', marker='x', linestyle='-', color='r')


# Invert the y-axis (since smaller magnitudes are brighter)
plt.gca().invert_yaxis()

# Add labels and title
plt.xlabel('Time [d]')
plt.ylabel('Magnitude')
plt.title('Model and observed')

# Add a legend to distinguish the two datasets
plt.legend()

# Show the plot
plt.show()


PLOTTING THE LIGHT CURVES GENERATED USING RUBIN SIM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load data from the first file (e.g., 'u_magnitude_time.dat')
data1 = np.loadtxt('lc_ra4.66_dec-0.36_event1_filter_u.dat', comments='#', skiprows=1)
time1 = data1[:, 0]
magnitude1 = data1[:, 1]



# Create the plot
plt.figure(figsize=(10, 6))

# Plot the first dataset
plt.plot(time1, magnitude1, label='model', marker='o', linestyle='-', color='b')




# Invert the y-axis (since smaller magnitudes are brighter)
plt.gca().invert_yaxis()

# Add labels and title
plt.xlabel('Time [d]')
plt.ylabel('Magnitude')
plt.title('u and g Filter Magnitudes vs Time')

# Add a legend to distinguish the two datasets
plt.legend()

# Show the plot
plt.show()
